In [69]:
#必要なパッケージのインポート

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import reservoirpy as rpy

from scipy.integrate import solve_ivp
import pandas as pd
from reservoirpy.observables import nrmse, rsquare

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import os

rpy.verbosity(0)

from reservoirpy.nodes import Reservoir, Ridge
from reservoirpy.datasets import mackey_glass

# just a little tweak to center the plots, nothing to worry about
from IPython.core.display import HTML
HTML("""
<style>
.img-center {
    display: block;
    margin-left: auto;
    margin-right: auto;
    }
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
    }
</style>
""")

rpy.set_seed(42)

%time


CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.86 µs


In [70]:
import numpy as np
import pandas as pd

# CSVファイルにデータを保存
filename_with_force = f'/Users/manqueenmannequin/mygit/B4thesis_public/mywork/program/VDP/VDP_analysis/generate_data/data/VDP_-7.csv'

# CSVファイルを読み込む
data_loaded_with_force = pd.read_csv(filename_with_force)

# CSVから値を抽出してNumpy配列に格納
X = data_loaded_with_force[['x', 'y', 'P_shifted']].values

time_series = X[:1000]
print(time_series.shape)


(1000, 3)


In [71]:
m = 2
tau = 1
R = 0.5

In [72]:
def delay_reconstruction_multi(time_series, m, tau):
    N = len(time_series)
    Y = np.array([time_series[i:i + m * tau:tau] for i in range(N - (m - 1) * tau)])
    return N, Y.reshape(-1, m * time_series.shape[1])

In [73]:
def count_similar_points(Y, R):
    N = len(Y)
    Ni = np.zeros(N)

    for i in range(N):
        distances = np.linalg.norm(Y - Y[i], ord=np.inf, axis=1)
        Ni[i] = np.sum(distances < R) - 1  # 自分自身を除く
    
    return Ni

In [74]:
def compute_phi(Ni, N, m):
    return np.mean(np.log(Ni[Ni > 0])) / (N - m + 1)

In [75]:
def approximate_entropy(time_series, m, tau, R):
    N, Y = delay_reconstruction_multi(time_series, m, tau)
    Ni_m = count_similar_points(Y, R)
    phi_m = compute_phi(Ni_m, N, m)

    N, Y = delay_reconstruction_multi(time_series, m + 1, tau)
    Ni_m1 = count_similar_points(Y, R)
    phi_m1 = compute_phi(Ni_m1, N, m + 1)

    return phi_m - phi_m1


In [76]:
approximate_entropy(time_series, m, tau, R)

4.493005862714339e-05